## --------------------------
## Get data
## --------------------------

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

processedDataStorePath = '../processedData/'

In [3]:
X_train_labeled = np.load(processedDataStorePath+'X_train_labeled.npy')
y_train_labeled = np.load(processedDataStorePath+'y_train_labeled.npy')
X_trainA_labeled = np.load(processedDataStorePath+'X_trainA_labeled.npy')
# y_trainA_labeled = np.load(processedDataStorePath+'y_trainA_labeled.npy')
X_trainB_labeled = np.load(processedDataStorePath+'X_trainB_labeled.npy')
# y_trainB_labeled = np.load(processedDataStorePath+'y_trainB_labeled.npy')
X_train_unlabeled = np.load(processedDataStorePath+'X_train_unlabeled.npy')
X_test = np.load(processedDataStorePath+'X_test.npy')
y_test = np.load(processedDataStorePath+'y_test.npy')

In [4]:
fa=[0, 2, 7, 8, 4]
fb=[1, 5, 9, 3, 6]
def splitFeatures(dataSet, featureList):
    return (dataSet.T[featureList]).T
X_testA = splitFeatures(X_test, fa)
X_testB = splitFeatures(X_test, fb)
X_trainA_unlabeled = splitFeatures(X_train_unlabeled, fa)
X_trainB_unlabeled = splitFeatures(X_train_unlabeled, fb)

## --------------------------
## Testing
## --------------------------

In [5]:
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from sklearn.tree import DecisionTreeClassifier
def testDecisonTree(X_train,y_train,X_test,y_test):
    print(X_train.shape)
    dt = DecisionTreeClassifier(random_state=0,max_depth=3,max_leaf_nodes=2)
    dt.fit(X_train,y_train)
    y_pred = dt.predict(X_test)
    print('decision tree:',accuracy_score(y_test,y_pred),'\n')
    return dt,accuracy_score(y_test,y_pred)

### self training

In [6]:
#base model
dt,acc = testDecisonTree(X_train_labeled,y_train_labeled,X_test,y_test)

(384, 10)
decision tree: 0.21291866028708134 



In [7]:
import heapq

# return all with same highest confidence
def getIndexWithHighConfidence(data,p):
    print('=========================================')
    print('=========================================')
    largestList=[]
    smallestList=[]
    largestValue = heapq.nlargest(1,data)[0]
    print('largest value: ', largestValue)
    smallestValue = heapq.nsmallest(1,data)[0]
    print('smallest value: ', smallestValue)
#     if largestValue>.7:
    largestList = [idx for idx, val in enumerate(data) if abs(val-largestValue)<=p]
#         if (len(largestList)>20):
#             random.shuffle(largestList)
#             largestList=largestList[:20]
    print('len of largest list: ', len(largestList))
#     if smallestValue<.3:
    smallestList = [idx for idx, val in enumerate(data) if abs(val-smallestValue)<=p]
#         if (len(smallestList)>20):
#             random.shuffle(smallestList)
#             smallestList=smallestList[:20]
    print('len of smallest list: ', len(smallestList))
    indexList = largestList + smallestList

    print('=========================================')
    print('=========================================')
    
    indexList.sort()
    indexList.reverse()
    return indexList

In [8]:
prob = 0.001
a=getIndexWithHighConfidence(dt.predict(X_test),prob)


largest value:  10.0
smallest value:  7.0
len of largest list:  277
len of smallest list:  141


In [9]:
from matplotlib import pyplot as plt
def drawLine(data):
    dataRange = [i for i in range(1,len(data)+1)]
    df = pd.DataFrame({'x':dataRange, 'y':data})
    plt.plot( 'x', 'y', data=df, marker='o', color='mediumvioletred')
    plt.show()


accuracy=[]
prob = 0.002

dt,acc = testDecisonTree(X_train_labeled,y_train_labeled,X_test,y_test)
accuracy.append(acc)

for i in range(100):

    y_pseudo = np.array(dt.predict(X_test))
    
    X_new=[]
    y_new=[]

    for index in getIndexWithHighConfidence(y_pseudo,prob):
        X_new.append(X_train_unlabeled[index])
        y_new.append(y_pseudo[index])
        X_train_unlabeled=np.delete(X_train_unlabeled,index,0) # 0 is row

    X_new=np.array(X_new)
    y_new=np.array(y_new)
    print('number of newly added: ',len(y_new))

    X_train_labeled = np.concatenate((X_train_labeled,X_new),axis=0)
    y_train_labeled = np.concatenate((y_train_labeled,y_new),axis=0)
    
    if i%10==0:
        prob=prob*1.5

    dt,acc = testDecisonTree(X_train_labeled,y_train_labeled,X_test,y_test)
    accuracy.append(acc)
    
drawLine(accuracy)

(384, 10)
decision tree: 0.21291866028708134 

largest value:  10.0
smallest value:  7.0
len of largest list:  277
len of smallest list:  141
number of newly added:  418
(802, 10)
decision tree: 0.1507177033492823 

largest value:  10.0
smallest value:  10.0
len of largest list:  418
len of smallest list:  418
number of newly added:  836
(1638, 10)
decision tree: 0.1507177033492823 

largest value:  10.0
smallest value:  10.0
len of largest list:  418
len of smallest list:  418
number of newly added:  836
(2474, 10)
decision tree: 0.1507177033492823 

largest value:  10.0
smallest value:  10.0
len of largest list:  418
len of smallest list:  418
number of newly added:  836
(3310, 10)
decision tree: 0.1507177033492823 

largest value:  10.0
smallest value:  10.0
len of largest list:  418
len of smallest list:  418


IndexError: index 389 is out of bounds for axis 0 with size 389

In [ ]:
dta = testDecisonTree(X_trainA_labeled,y_train_labeled,X_testA,y_test)

In [ ]:
dtb = testDecisonTree(X_trainB_labeled,y_train_labeled,X_testB,y_test)

In [ ]:
# create a copy for testing purpose
# X_trainval_1 = 
# X_unlabeled_1=X_unlabeled[:]
# y_unlabeled_1=y_unlabeled[:]